## Datamart creation

#### 1) Load libraries and configure Snowflake connector

In [2]:
pip install snowflake.connector.python

  Obtaining dependency information for snowflake.connector.python from https://files.pythonhosted.org/packages/21/ad/f173bc54e55744fba26abc9777ab35b0247cf112861ae8b16e1613139613/snowflake_connector_python-3.5.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.9/912.9 kB 24.3 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import snowflake.connector
import pandas as pd
import numpy as np
from datetime import datetime
from Module_6_Homework_Support_Functions import create_database_table
from Module_6_Homework_Support_Functions import create_initial_surrogate_key_mapping_table

In [58]:
# Connect to Snowflake
connection = snowflake.connector.connect(
    user = 'ziqilong',
    password = 'Ch@ngeme1',
    account = 'UTB94111'
)
cur = connection.cursor()

#### 2)  Create and Populate the Star Schema

In [3]:
# Read the data from the HW6 database into the dataframe Products
# Product Dimention
cur.execute("SELECT P.ProductID, P.ProductName, C.CategoryName \
               FROM HW6.Public.PRODUCTS P \
               LEFT JOIN HW6.Public.CATEGORIES C \
               WHERE P.CATEGORYID = C.CATEGORYID")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
PRODUCTS = pd.DataFrame(rows, columns = columns)
PRODUCTS, PRODUCTS_surrogate_key_mapping = create_initial_surrogate_key_mapping_table(table = PRODUCTS,
                                                                              surrogate_key_name = 'PRODUCTS_KEY', 
                                                                              start_date = datetime(2000,1,1),
                                                                              end_date = datetime(2099,12,31))

In [4]:
PRODUCTS

,PRODUCTS_KEY,PRODUCTID,PRODUCTNAME,CATEGORYNAME
0,0,1,Chai,Beverages
1,1,2,Chang,Beverages
2,2,3,Aniseed Syrup,Condiments
3,3,4,Chef Antons Cajun Seasoning,Condiments
4,4,5,Chef Antons Gumbo Mix,Condiments
...,...,...,...,...
72,72,73,R d Kaviar,Seafood
73,73,74,Longlife Tofu,Produce
74,74,75,Rh nbr u Klosterbier,Beverages
75,75,76,Lakkalik ri,Beverages


In [5]:
create_database_table(PRODUCTS, 'PRODUCTS_KEY', 'HW6_datamart', 'products_dim')

In [6]:
create_database_table(PRODUCTS_surrogate_key_mapping, 'PRODUCTS_KEY', 'HW6_datamart', 
                      'PRODUCTS_surrogate_mapping_table' )

In [8]:
# Customer Dimension
cur.execute("SELECT C.CustomerID, C.ContactName, C.Country \
               FROM HW6.Public.CUSTOMERS C")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
CUSTOMERS = pd.DataFrame(rows, columns = columns)
CUSTOMERS, CUSTOMERS_surrogate_key_mapping = create_initial_surrogate_key_mapping_table(table = CUSTOMERS,
                                                                              surrogate_key_name = 'CUSTOMERS_KEY', 
                                                                              start_date = datetime(2000,1,1),
                                                                              end_date = datetime(2099,12,31))

In [9]:
create_database_table(CUSTOMERS, 'CUSTOMERS_KEY', 'HW6_datamart', 'customers_dim')
create_database_table(CUSTOMERS_surrogate_key_mapping, 'CUSTOMERS_KEY', 'HW6_datamart', 
                      'CUSTOMERS_surrogate_mapping_table' )

In [10]:
# Shipper Dimension
cur.execute("SELECT S.ShipperID, S.CompanyName \
               FROM HW6.Public.SHIPPERS S")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
SHIPPERS = pd.DataFrame(rows, columns = columns)
SHIPPERS, SHIPPERS_surrogate_key_mapping = create_initial_surrogate_key_mapping_table(table = SHIPPERS,
                                                                              surrogate_key_name = 'SHIPPERS_KEY', 
                                                                              start_date = datetime(2000,1,1),
                                                                              end_date = datetime(2099,12,31))

In [11]:
create_database_table(SHIPPERS, 'SHIPPERS_KEY', 'HW6_datamart', 'shippers_dim')
create_database_table(SHIPPERS_surrogate_key_mapping, 'SHIPPERS_KEY', 'HW6_datamart', 
                      'SHIPPERS_surrogate_mapping_table' )

In [12]:
# Employee Dimension
cur.execute("SELECT E.EmployeeID, E.LastName, E.FirstName, E.City, E.Country\
               FROM HW6.Public.EMPLOYEES E")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
EMPLOYEES = pd.DataFrame(rows, columns = columns)
EMPLOYEES, EMPLOYEES_surrogate_key_mapping = create_initial_surrogate_key_mapping_table(table = EMPLOYEES,
                                                                              surrogate_key_name = 'EMPLOYEES_KEY', 
                                                                              start_date = datetime(2000,1,1),
                                                                              end_date = datetime(2099,12,31))

In [13]:
create_database_table(EMPLOYEES, 'EMPLOYEES_KEY', 'HW6_datamart', 'employees_dim')
create_database_table(EMPLOYEES_surrogate_key_mapping, 'EMPLOYEES_KEY', 'HW6_datamart', 
                      'EMPLOYEES_surrogate_mapping_table' )

In [18]:
# Date Dimension
cur.execute("SELECT ORDERDATE AS date\
            FROM HW6.Public.ORDERS O")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
DATES = pd.DataFrame(rows, columns = columns)
DATES

,DATE
0,1996-07-04
1,1996-07-05
2,1996-07-08
3,1996-07-08
4,1996-07-09
...,...
825,1998-05-05
826,1998-05-06
827,1998-05-06
828,1998-05-06


In [19]:
from datetime import datetime
from datetime import date

In [22]:
DATES['DATE'] = pd.to_datetime(DATES['DATE'])
DATES['day_of_week'] = DATES['DATE'].dt.day_name()
DATES['week_of_year'] = DATES['DATE'].dt.isocalendar().week
DATES['month_of_year'] = DATES['DATE'].dt.month
DATES['year'] = DATES['DATE'].dt.year

In [23]:
# Drop any duplicate rows (transactions that occurred at the same date/time)
DATES = DATES.drop_duplicates()

In [24]:
create_database_table(DATES, 'DATE', 'HW6_datamart', 'date_dim')

In [27]:
DATES, DATES_surrogate_key_mapping = create_initial_surrogate_key_mapping_table(table = DATES,
                                                                              surrogate_key_name = 'DATE_key', 
                                                                              start_date = datetime(2000,1,1),
                                                                              end_date = datetime(2099,12,31))

In [28]:
create_database_table(DATES_surrogate_key_mapping, 'DATE', 'HW6_datamart', 
                      'DATES_surrogate_mapping_table' )

In [40]:
# Fact table
cur.execute("SELECT O.ORDERID, O.ORDERDATE, O.CustomerID, O.EmployeeID, \
            P.PRODUCTID, O.SHIPVIA, OD.UnitPrice, OD.Quantity, OD.Discount, \
            (OD.UnitPrice*OD.Quantity-OD.Discount) as TotalRevenue \
            FROM HW6.Public.ORDERS O\
            LEFT JOIN HW6.Public.OrderDetails OD ON O.ORDERID = OD.ORDERID\
            LEFT JOIN HW6.Public.Products P ON P.PRODUCTID = OD.PRODUCTID")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
Initial_Fact_Table = pd.DataFrame(rows, columns = columns)
Initial_Fact_Table

,ORDERID,ORDERDATE,CUSTOMERID,EMPLOYEEID,PRODUCTID,SHIPVIA,UNITPRICE,QUANTITY,DISCOUNT,TOTALREVENUE
0,10248,1996-07-04,VINET,5,11,3,14,12,0,168
1,10248,1996-07-04,VINET,5,42,3,10,10,0,100
2,10248,1996-07-04,VINET,5,72,3,35,5,0,175
3,10249,1996-07-05,TOMSP,6,14,1,19,9,0,171
4,10249,1996-07-05,TOMSP,6,51,1,42,40,0,1680
...,...,...,...,...,...,...,...,...,...,...
2150,11077,1998-05-06,RATTC,1,64,2,33,2,0,66
2151,11077,1998-05-06,RATTC,1,66,2,17,1,0,17
2152,11077,1998-05-06,RATTC,1,73,2,15,2,0,30
2153,11077,1998-05-06,RATTC,1,75,2,8,4,0,32


In [41]:
#  Read the surrogate key mapping tables
cur.execute("Select * FROM HW6_datamart.Public.PRODUCTS_surrogate_mapping_table")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
products_surrogate_mapping_table = pd.DataFrame(rows, columns = columns)

cur.execute("Select * FROM HW6_datamart.Public.CUSTOMERS_surrogate_mapping_table")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
customers_surrogate_mapping_table = pd.DataFrame(rows, columns = columns)

cur.execute("Select * FROM HW6_datamart.Public.SHIPPERS_surrogate_mapping_table")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
shippers_surrogate_mapping_table = pd.DataFrame(rows, columns = columns)

cur.execute("Select * FROM HW6_datamart.Public.EMPLOYEES_surrogate_mapping_table")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
employees_surrogate_mapping_table = pd.DataFrame(rows, columns = columns)

In [51]:
Orders_Fact_Table1 = Initial_Fact_Table
Orders_Fact_Table1['PRODUCTS_KEY'] = Orders_Fact_Table['PRODUCTID'].map(products_surrogate_mapping_table.\
                                                                       set_index('PRODUCTID')['PRODUCTS_KEY'])
Orders_Fact_Table1['CUSTOMERS_KEY'] = Orders_Fact_Table['CUSTOMERID'].map(customers_surrogate_mapping_table.\
                                                                         set_index('CUSTOMERID')['CUSTOMERS_KEY'])
Orders_Fact_Table1['SHIPPERS_KEY'] = Orders_Fact_Table['SHIPVIA'].map(shippers_surrogate_mapping_table.\
                                                                     set_index('SHIPPERID')['SHIPPERS_KEY'])
Orders_Fact_Table1['EMPLOYEES_KEY'] = Orders_Fact_Table['EMPLOYEEID'].map(employees_surrogate_mapping_table.\
                                                                         set_index('EMPLOYEEID')['EMPLOYEES_KEY'])
Orders_Fact_Table1['ORDERDATE'] = pd.to_datetime(Orders_Fact_Table1['ORDERDATE'])
Orders_Fact_Table1

,ORDERID,ORDERDATE,CUSTOMERID,EMPLOYEEID,PRODUCTID,SHIPVIA,UNITPRICE,QUANTITY,DISCOUNT,TOTALREVENUE,PRODUCTS_KEY,CUSTOMERS_KEY,SHIPPERS_KEY,EMPLOYEES_KEY
0,10248,1996-07-04,VINET,5,11,3,14,12,0,168,10,84,2,4
1,10248,1996-07-04,VINET,5,42,3,10,10,0,100,41,84,2,4
2,10248,1996-07-04,VINET,5,72,3,35,5,0,175,71,84,2,4
3,10249,1996-07-05,TOMSP,6,14,1,19,9,0,171,13,78,0,5
4,10249,1996-07-05,TOMSP,6,51,1,42,40,0,1680,50,78,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,11077,1998-05-06,RATTC,1,64,2,33,2,0,66,63,64,1,1
2151,11077,1998-05-06,RATTC,1,66,2,17,1,0,17,65,64,1,1
2152,11077,1998-05-06,RATTC,1,73,2,15,2,0,30,72,64,1,1
2153,11077,1998-05-06,RATTC,1,75,2,8,4,0,32,74,64,1,1


In [52]:
Orders_Fact_Table1, order_surrogate_key_mapping = create_initial_surrogate_key_mapping_table(\
                                                                              table = Orders_Fact_Table1,
                                                                              surrogate_key_name = 'ORDERS_KEY', 
                                                                              start_date = datetime(2000,1,1),
                                                                              end_date = datetime(2099,12,31))

In [53]:
create_database_table(Orders_Fact_Table1, 'ORDERS_KEY', 'HW6_datamart', 'order_fact')
create_database_table(order_surrogate_key_mapping, 'ORDERS_KEY', 'HW6_datamart', 'order_surrogate_mapping_table')

#### 3A)  Test Query 1

In [70]:
cur.execute("SELECT SUM(TotalRevenue) AS TotalRevenue_tofu \
             FROM HW6_DATAMART.Public.Order_Fact F \
             LEFT JOIN HW6_DATAMART.Public.DATE_DIM d ON F.ORDERDATE = d.DATE\
             LEFT JOIN HW6_DATAMART.Public.PRODUCTS_DIM p ON F.PRODUCTID = p.PRODUCTID\
             WHERE d.Year = '1996' AND d.Month_of_year = '7'\
                   AND p.PRODUCTNAME = 'Tofu' ")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
Revenue_tofu = pd.DataFrame(rows, columns = columns)
Revenue_tofu 

,TOTALREVENUE_TOFU
0,171


#### 3B)  Test Query 2

In [72]:
cur.execute("SELECT e.lastname, e.firstname, e.country, SUM(TotalRevenue) AS TotalOrderValue, \
             RANK() OVER (PARTITION BY COUNTRY ORDER BY TotalOrderValue DESC) AS value_rank\
             FROM HW6_DATAMART.Public.Order_Fact F\
             LEFT JOIN  HW6_DATAMART.Public.EMPLOYEES_DIM e ON e.employeeid = F.employeeid\
             GROUP BY e.employeeid,e.lastname,e.firstname,e.country\
             ORDER BY value_rank\
             LIMIT 2 ")
rows = cur.fetchall()
columns = [column[0] for column in cur.description]
EmployeeOrderValue = pd.DataFrame(rows, columns = columns)
EmployeeOrderValue 

,LASTNAME,FIRSTNAME,COUNTRY,TOTALORDERVALUE,VALUE_RANK
0,King,Robert,UK,141489,1
1,Peacock,Margaret,USA,250451,1
